In [ ]:
# Import necessary libraries
import os, sys, shutil, glob, argparse
import numpy as np
from PIL import Image
from lxml import etree
from scipy.io import loadmat
from datetime import datetime

python_version = sys.version_info.major


###########################################################
##########       IMDB-WIKI to JSON Conversion       ##########
###########################################################

abs_path = 'datasets'
wiki_path = os.path.join(abs_path, 'IMDB-WIKI/')
json_path = os.path.join(abs_path, 'JSON/')
annotations_file = os.path.join(wiki_path, 'wiki.mat')
db = "wiki"

def calc_age(taken, dob):
    birth = datetime.fromordinal(max(int(dob) - 366, 1))
    # assume the photo was taken in the middle of the year
    if birth.month < 7:
        return taken - birth.year
    else:
        return taken - birth.year - 1

def make_json_directories():
    """
    Definition: Make directories for json images and annotations.
        Removes previously created json image and annotation directories.
    Returns: None
    """
    if os.path.exists(json_path):
        if python_version == 3:
            prompt = input('Directory already exists. Overwrite? (yes, no): ')
        else:
            prompt = raw_input('Directory already exists. Overwrite? (yes, no): ')
        if prompt == 'no':
            exit(0)
        shutil.rmtree(json_path)
    os.makedirs(json_path)
    os.makedirs(json_path + "images")
    os.makedirs(json_path + "annotations")
def copy_images_json():
    """
    Definition: Copy all images from 00-99 directories of wiki dataset
         to image directory in in JSON/images folder.
    """
    
    for i in range(100):
        if i < 10:
            for filename in glob.glob(os.path.join(wiki_path + "0"+str(i), "*.*")):
                shutil.copy(filename, json_path + "images/")
        if i >= 10:
            for filename in glob.glob(os.path.join(wiki_path + str(i), "*.*")):
                shutil.copy(filename, json_path + "images/")
    
def parse():
    
    """
        Definition: Make annotations directory for wiki annotations and populate it with separate ann files.
        Returns: None
    """
   
    meta = loadmat(annotations_file)
    full_path = meta[db][0, 0]["full_path"][0]
    dob = meta[db][0, 0]["dob"][0]  # Matlab serial date number
    gender = meta[db][0, 0]["gender"][0]
    name = meta[db][0, 0]["name"][0]
    face_location = meta[db][0, 0]["face_location"][0]
    photo_taken = meta[db][0, 0]["photo_taken"][0]  # year
    face_score = meta[db][0, 0]["face_score"][0]
    second_face_score = meta[db][0, 0]["second_face_score"][0]
    age = [calc_age(photo_taken[i], dob[i]) for i in range(len(dob))]
    
    objects = []   
    object_info = {}
    ind = -1
    coords = []
    for i in range(len(dob)):
        ind += 1
        if ind < 1000:
            object_info['filename'] = str(full_path[i]).split("/")[1].split("'")[0]
            object_info['objects'] = []
            face_info = {'class_name':'face'}
            for row in face_location[i]:
                for j in row:
                    coords.append(int(j))
                face_info['bounding_box'] = coords
                coords = []
            
            face_info['gender'] = gender[i]
            face_info['age'] = age[i]
            object_info['objects'].append(face_info)
            objects.append(object_info)
        for j in objects:
            f= open(json_path+'annotations/'+j["filename"].split(".jpg")[0]+'.json',"w+")
            f.write(str(j))
            f.close()
    else:
            return

def json():
    make_json_directories()
    copy_images_json()
    parse()
    
json()